ETL
Con el fin de facilitar el análisis, el proceso de ETL se realizará de manera local a través de un notebook de python 2, el cuál trabajaremos por medio de dataframes.

In [ ]:
Extracción
Simplificado en la descarga del archivo desde el drop box, descomprimirlo y trabajarlo de manera local

In [2]:
#importar librerías necesarias
import pandas as pd

#Lectura de data, agregándo encabezados para facilitar su manipulación
df = pd.read_csv(r'input\bbdd prueba corp.csv', 
                 sep='\073',
                 names = ["PATENTE","MARCA","MODELO","AGNO","ID_CLIENTE","COMUNA","REGION","SEXO","ACTIVIDAD","TASACION","FEC_TRANSFERENCIA","COLOR2","EDAD","VIGENCIA"]) 

#muestra de la data carga con cabecera
df.head()

,PATENTE,MARCA,MODELO,AGNO,ID_CLIENTE,COMUNA,REGION,SEXO,ACTIVIDAD,TASACION,FEC_TRANSFERENCIA,COLOR2,EDAD,VIGENCIA
0,393A4B4C-085,TOYOTA,RAV 4,2015,50838335,TEMUCO,09,M,NaN,"11947500,0",NaN,BLANCO,NaN,N
1,B1F3DB7E-F67,MAZDA,CX 5,2016,46322649,VILLA ALEMANA,05,M,NaN,NaN,NaN,NEGRO,38.0,S
2,F6DBB2E6-A76,GREAT WALL,HAVAL NEW H3 2.0,2014,36226810,ANTOFAGASTA,DE ANTOFAGASTA,NaN,NaN,NaN,NaN,NEGRO,49.0,S
3,EBCF63CD-99D,JEEP,COMPASS SPORT 2.4,2013,43482783,NaN,NaN,NaN,NaN,NaN,20160418.0,GRIS,40.0,S
4,FF38B368-B0F,SUBARU,FORESTER 2.0,2017,25657273,NaN,NaN,NaN,NaN,NaN,NaN,GRIS,62.0,S


Transformación
Ocuparemos los formatos mas idóneos para el tipo de dato además de binarizar algunas columnas para facilitar su análisis

In [5]:
from datetime import datetime
import numpy as np

print(len(df))
print((df.notnull().sum()/len(df)*100).round(2))

#transformar FEC_TRANSFERENCIA a formato fecha
df['SM_FEC_TRANSFERENCIA'] = pd.to_datetime(df['FEC_TRANSFERENCIA'], format='%Y%m%d').dt.strftime("%Y-%m-%d")

#binarizar variable vigencia
df['VIGENCIA_BIN'] = np.where(df['VIGENCIA'] == 'S', 1, 0)



1006969
S    688992
N    317977
Name: VIGENCIA, dtype: int64
PATENTE              100.00
MARCA                 99.93
MODELO                99.97
AGNO                 100.00
ID_CLIENTE           100.00
COMUNA                57.07
REGION                59.05
SEXO                  12.18
ACTIVIDAD             19.64
TASACION              60.48
FEC_TRANSFERENCIA     33.88
COLOR2                98.75
EDAD                  71.85
VIGENCIA             100.00
SM_REGION             59.05
dtype: float64


Determinamos el porcentaje de datos poblados por variable y en base a su porcentaje determinaremos que variables consideraremos para analizar, de manera de trabajar con la mayor cantidad de datos posibles.

Para nuestro análisis sólo consideraremos variables cuyo porcentaje de datos poblados sea mayor al 58%, descartando para el análisis: COMUNA, SEXO, ACTIVIDAD.

Para la variable REGION normalizaremos su distribución.

In [6]:
#rellenar de 0 regiones con números menor a 0
df['REGION_LPAD'] = df['REGION'].str.pad(width=2, side='left', fillchar='0')

#normalizar regiones a números
df.loc[df['REGION_LPAD'].str.contains('ÑUBLE|NUBLE|16', na=False), 'SM_REGION'] = '16'
df.loc[df['REGION_LPAD'].str.contains('ARICA|PARINACOTA|15', na=False), 'SM_REGION'] = '15'
df.loc[df['REGION_LPAD'].str.contains('RIOS|14', regex=True, na=False), 'SM_REGION'] = '14'
df.loc[df['REGION_LPAD'].str.contains('METROPOLITANA|13', na=False), 'SM_REGION'] = '13'
df.loc[df['REGION_LPAD'].str.contains('ANTARTICA|MAGALLANES|12', na=False), 'SM_REGION'] = '12'
df.loc[df['REGION_LPAD'].str.contains('AYSEN|CARLOS|IBANEZ|11', na=False), 'SM_REGION'] = '11'
df.loc[df['REGION_LPAD'].str.contains('LAGOS|10', na=False), 'SM_REGION'] = '10'
df.loc[df['REGION_LPAD'].str.contains('ARAUCANIA|09', na=False), 'SM_REGION'] = '09'
df.loc[df['REGION_LPAD'].str.contains('BIO|08', na=False), 'SM_REGION'] = '08'
df.loc[df['REGION_LPAD'].str.contains('MAULE|07', na=False), 'SM_REGION'] = '07'
df.loc[df['REGION_LPAD'].str.contains('HIGGINS|06', na=False), 'SM_REGION'] = '06'
df.loc[df['REGION_LPAD'].str.contains('VALPARAISO|05', na=False), 'SM_REGION'] = '05'
df.loc[df['REGION_LPAD'].str.contains('ARAUCANIA|04', na=False), 'SM_REGION'] = '04'
df.loc[df['REGION_LPAD'].str.contains('COQUIMBO|03', na=False), 'SM_REGION'] = '03'
df.loc[df['REGION_LPAD'].str.contains('ANTOFAGASTA|02', na=False), 'SM_REGION'] = '02'
df.loc[df['REGION_LPAD'].str.contains('TARAPACA|01', na=False), 'SM_REGION'] = '01'

df['SM_REGION'].value_counts().sort_index()

01      6197
02     29254
03     13457
04     16636
05     48579
06     19009
07     15544
08     45589
09      9024
10     14855
11      2007
12      5868
13    302218
14      5459
15      2385
Name: SM_REGION, dtype: int64

Para un futuro análisis realizaremos una división de las regiones por Zonas (Norte, Centro, Sur)

In [11]:
def flag_df(df):

    if df['SM_REGION'] in ('15','01','02','03','04'):
        return 'NORTE'
    elif df['SM_REGION'] in ('05','06','07','08'):
        return 'CENTRO'
    elif df['SM_REGION'] in ('09','10','11','12','14'):
        return 'SUR'
    elif df['SM_REGION'] == '13':
        return 'RM'
    else:
        return np.nan

df['ZONA'] = df.apply(flag_df, axis = 1)

df['ZONA'].value_counts()

df['ZONA_NORTE'] = np.where(df['ZONA'] == 'NORTE', 1, 0)
df['ZONA_CENTRO'] = np.where(df['ZONA'] == 'CENTRO', 1, 0)
df['ZONA_SUR'] = np.where(df['ZONA'] == 'SUR', 1, 0)
df['RM'] = np.where(df['ZONA'] == 'RM', 1, 0)


NameError: name 'f' is not defined

Guardaremos la nueva tabla enriquecida para su análisis

In [19]:
del df['REGION']
del df['VIGENCIA']
del df['REGION_LPAD']
del df['FEC_TRANSFERENCIA']

df.to_csv(r'output\bbdd_prueba_corp_enriched.csv', na_rep='NaN', index=False)